# House Sales Analysis in NorthWestern county

## 1. Business Understanding
### a) Introduction

House sales began in 1890s in the United States and since then its been growing all over the world and agencies started to form to enhance and ease the house selling process.Last year the revenue was estimated to be $4.25M with prospects of growth as time goes by. House sales are mainly influenced by the number of bedrooms, bathrooms, the year built, square footage and whether renovations are done or not among other factors.

In this case the Northwest agencies aim to address the need of providing homeowners with accurate and actionable advice on how home renovations can potentially increase the estimated value of their properties and by what amount. By understanding the relationship between various renovation factors and house prices, the agencies can be able to guide homeowners in making informed decisions about renovations, which will ultimately lead to maximization of return on their investment which will enable them sell their homes at optimal prices.

### b) Problem statement

The real estate industry faces the challenge of providing homeowners with reliable information about how various home renovation factors impact the estimated value of their homes. Our project addresses this problem by utilizing data analysis and regression modeling to identify key factors that affect house prices in a northwestern county. By understanding these factors, we can provide recommendations and insights to stakeholders on how to effectively advise homeowners on renovations that can potentially increase the value of their properties.

### c) Main Objective

The main objective of this project is to develop a predictive model that estimates house prices based on various features such as the number of bedrooms and bathrooms, square footage, and year built. By building a robust regression model, we aim to accurately predict house prices and provide stakeholders with valuable insights into the factors driving price fluctuations.

### d) Metric of success

The success of our project will be evaluated based on the model's performance in predicting house prices. We will use evaluation metrics such as the coefficient of determination (R-squared) and root mean square error (RMSE) to assess the model's accuracy. A higher R-squared value and lower RMSE indicate a more successful model in capturing the variations in house prices.

### e) Specific Objectives

- Explore and preprocess the King County House Sales dataset, including handling missing values, transforming features, and encoding categorical variables.
- Perform exploratory data analysis to gain insights into the distribution and relationships between different features and the target variable.
- Conduct feature selection to identify the most influential factors that affect house prices and eliminate irrelevant or redundant features.
- Build multiple linear regression models with different combinations of features and evaluate their performance using appropriate metrics.
- Interpret the results of the final regression model, including the coefficients of the selected features and their implications on house prices.
- Provide recommendations to stakeholders based on the insights gained from the modeling process, suggesting specific renovation factors that homeowners can focus on to increase the estimated value of their properties.

##  2. Data Understanding

The data used for this project is the King County House Sales dataset. The dataset contains information on house sales in a northwestern county, including various features such as the number of bedrooms and bathrooms, square footage, location, and other relevant details.The dataset is suitable for the project as it provides comprehensive information about house sales and house features, which allows for analysis and modeling to understand the relationship between these features and the sale prices of the houses.

The dataset consists of a substantial number of records, with each record representing a house sale. It includes information on multiple features, such as bedrooms, bathrooms, square footage, and more. To gain insights into the dataset, we will present descriptive statistics for all the features used in the analysis. These statistics will include measures of central tendency  and dispersion to provide an overview of the distribution and variability of the data.

The features included in the analysis are selected based on their relevance and potential impact on house prices. Features such as the number of bedrooms and bathrooms, square footage, and location are commonly considered important factors affecting house prices. By including these features in the analysis, we aim to capture the significant aspects that contribute to the variation in house prices and provide valuable insights to homeowners seeking advice on home renovations.

Even though this dataset provides a rich source of information, it also has limitations which include absence features that could also influence house prices  such as proximity to public transportation, missing data in certain columns, and the inherent complexity of real estate market dynamics that cannot be fully captured by the dataset alone.


## 3. Data Preparation

This process involves cleaning, transforming, and organizing the data to ensure its suitability for analysis and modeling. 

- Importing relevant libraries
- Loading the dataset and checking it contains
- Dealing with missing data
- Checking and removing duplicates
- Handling outliers
- Feature scaling and normalization using z-scores
- Encoding categoriacl variables using one-hot encoding
- Exploring the dataset to identify opportunities for creating new features that may enhance the predictive power of the model 
- Splitting the dataset into training and test sets


### 3.1 Importing libraries

In [1]:
# importing libraries
import pandas as pd
import numpy as np 
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats

### 3.2 Loading the dataset

In [26]:
# reading the house dataset and previewing the last five outputs to check what each column contains
data = pd.read_csv("data/kc_house_data.csv")
data.tail()

In [3]:
# getting an overview of the dataset, including the number of non-null values and the data types of each column
print(data.info())
# getting the number of rows and columns in the data.
print(data.shape)

The dataset has 21597 rows and 21 columns

In [4]:
# checking the datatypes of each columns
data.dtypes

In [5]:
# generating summary statistics for the numerical columns in the dataset
data.describe()

Most houses have an average of 3 bedrooms,2 bathrooms and were built between 1970 and 2015

## Data Cleaning

### Handling missing values

We will check to see which columns have missing values and fill them using .fillna()

In [6]:
# checking the proportion of missing values per column
data.isna().sum()/len(data)

Three columns have missing values but we will only work with waterfront since it will be used in analysis and modelling

In [7]:
# since waterfront has missing values, we first check the value counts
data['waterfront'].value_counts()

In [8]:
# filling in missing values in the 'waterfront' column with the string 'NO'
data['waterfront'] = data['waterfront'].fillna('NO')
# getting an overview of the dataset after filling the missing values in Waterfront
data.info()

We filled the missing values in waterfront instead of removing them since we want each column to have the same number of rows.
we can now see that the waterfront column has the same values as the other columns we will be using.

### Dropping Columns

Dropping columns that will not be used during modelling 

In [9]:
# dropping columns that we will not be using in data analysis.
columns_to_drop = ['date', 'view', 'sqft_above', 'sqft_basement', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']
data.drop(columns=columns_to_drop, inplace=True)

In [10]:
# checking to see if the columns have been dropped.
print(data.info())

The columns have been dropped and we remaining with 11 columns that we will be working with.

### Handling duplicates

We will check if there are any duplicated values, drop them incase they are there and keep the first value

In [11]:
# checking if there are any duplicates
data.duplicated().any()

In [12]:
# checking the sum of duplicated ids
data['id'].duplicated().sum()
# dropping the duplicated values and keeping the first in id column
data = data.drop_duplicates(subset='id', keep='first')

In [13]:
# now checking the if there are any duplicated values
data.duplicated().any()

In [14]:
# checking the shape of the data to see if duplicated values have been dropped.
data.shape

As we can see the entry values changed from 21597 to 21420 meaning duplicated values were dropped

### Handling outliers

Firstly we will identify outliers, get visualizations when there are outliers, we will then go ahead and remove them and get visualizations to see the difference.

In [15]:
# Selecting the numerical columns in the dataset
numeric_columns = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'yr_built', 'floors']

# Calculating z-scores for the numerical columns
z_scores = (data[numeric_columns] - data[numeric_columns].mean()) / data[numeric_columns].std()

# defining a threshold for identifying outliers
threshold = 2

# Find the indices of outliers for each column
outlier_indices = (z_scores > threshold).any(axis=1)

# Extract the outlier rows from the dataset
outliers = data[outlier_indices]

# Plotting the outliers
for column in numeric_columns:
    plt.figure()
    plt.boxplot(data[column])
    plt.title(column)
    plt.show()


In [16]:
# Removing the outlier rows from the dataset
data_cleaned = data[~outlier_indices]

# Reset the index of the cleaned dataset
data_cleaned.reset_index(drop=True, inplace=True)

# Plot histograms of the cleaned dataset
data_cleaned[numeric_columns].hist(bins=20, figsize=(10, 8))
plt.tight_layout()
plt.show()

# Plot scatterplots of price against other numeric columns
sns.pairplot(data_cleaned, x_vars=['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot'], y_vars='price', height=5)
plt.tight_layout()
plt.show()



### One-Hot encoding

First, we check for categorical columns, then get the value counts and lastly perform encoding.

In [17]:
# checking for categorical columns
categorical_columns = data.select_dtypes(include=['object', 'category']).columns
categorical_columns

In [18]:
# exploring categorical variables,using value_counts() to get the count of each unique value in a column 
# using describe() to get summary statistics for categorical columns.
for column in categorical_columns:
    value_counts = data[column].value_counts()
    print(f"Value counts for {column}:\n{value_counts}\n")

# Summary statistics for categorical columns
print(data[categorical_columns].describe())

In [19]:
# one hot encoding to convert categoriacl values into binary'
data_encoded = pd.get_dummies(data, columns=categorical_columns, drop_first=True)
data_encoded.head()

 ### Feature scaling and normalization using z-scores
This is done to numerical features to standardize their values and ensure they are on a similar scale and  helps prevent any bias or undue influence that may arise from differences in the magnitude of numerical features

In [20]:
 # Computing the z-scores for the numerical features only
data_numeric = data[numeric_columns]
data_numeric = (data_numeric - data_numeric.mean()) / data_numeric.std()

# Combining the encoded categorical features with the transformed numerical features
data_encoded = pd.concat([data_encoded, data_numeric], axis=1)

# Checking the updated dataset
print(data_encoded.head())


### Correlation Analysis

Correlation analysis is a statistical technique used to measure the strength and direction of the linear relationship between two or more variables. It helps identify the degree of association between variables and provides insights into their interdependencies.


In [21]:
# Correlation analysis
correlation_matrix = data_cleaned[numeric_columns].corr()
print("\nCorrelation Matrix:")
print(correlation_matrix)

## Regression modelling

To perform regression modeling, we will split the dataset into features (X) and target variable (y), and then split them into training and testing sets. We will use the LinearRegression model from scikit-learn library to build the regression model.

### Baseline model

We going to create a simple regression model without using any modifications to establish a baseline performance metric that other models can compare aganaist

In [22]:
# Selecting the target variable and features
target = 'price'
features = ['sqft_living']

# Split the dataset into training and test sets
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the baseline linear regression model
baseline_model = LinearRegression()

# Fit the model on the training data
baseline_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = baseline_model.predict(X_test)

# Calculate the mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the MSE
print("\nBaseline Regression Model Evaluation:")
print(f"R-squared: {r2:.2f}")
print('Baseline Model MSE:', mse)

In [23]:
# Selecting the target variable and features
target = 'price'
features = ['bedrooms', 'bathrooms', 'sqft_living']

# Split the dataset into training and test sets
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the baseline linear regression model
baseline_model = LinearRegression()

# Fit the model on the training data
baseline_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = baseline_model.predict(X_test)

# Calculate the mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the MSE
print("\nBaseline Regression Model Evaluation:")
print(f"R-squared: {r2:.2f}")
print('Baseline Model MSE:', mse)


### Second Model using polynomial Transformation

This model is introduced to build on the baseline model, it will incorporate additional features such as polynomial transformation to capture non linear relationships to improve the performance of the model.

In [24]:

# Select the target variable and features
#X = data_encoded
#features = ['bedrooms', 'bathrooms', 'sqft_living']

# Split the dataset into training and test sets
#X = data[features]
#y = data[target]
X = data_encoded.drop(['price'], axis=1)
y = data_encoded['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a new feature using polynomial transformation
poly = PolynomialFeatures(degree=2)
X_poly_train = poly.fit_transform(X_train)

# Create a new model using the transformed features
model_two = LinearRegression()
model_two.fit(X_poly_train, y_train)

# Transform the test features
X_poly_test = poly.transform(X_test)

# Make predictions on the test data
y_pred = model_two.predict(X_poly_test)

# Calculate the mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the MSE
print("\nModel2 Regression Model Evaluation:")
print(f"R-squared: {r2:.2f}")
print('Model2 MSE:', mse)


The improvements in R^2 indicate that the new model explains a larger portion of the variance in the target variable compared to the baseline models. This means that the new model provides a better fit to the data and has the potential to make more accurate predictions. The reduction in MSE implies that the new model has smaller prediction errors, which can be valuable for stakeholders in terms of making informed decisions or pricing properties more accurately.

In [25]:
# Evaluate the baseline model
baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)
baseline_predictions = baseline_model.predict(X_test)
baseline_mse = mean_squared_error(y_test, baseline_predictions)
baseline_r2 = baseline_model.score(X_test, y_test)
print('Baseline Model MSE:', baseline_mse)
print('Baseline Model R^2:', baseline_r2)

# Evaluate the new model
model_two = LinearRegression()
model_two.fit(X_train_new, y_train)
new_predictions = model_twol.predict(X_test_new)
new_mse = mean_squared_error(y_test, new_predictions)
new_r2 = new_model.score(X_test_new, y_test)
print('Model Two mse:', new_mse)
print('Model Two R^2:', new_r2)

# Justification
if new_mse < baseline_mse:
    print('The new model improves on the baseline model in terms of MSE.')
else:
    print('The new model does not provide a significant improvement over the baseline model in terms of MSE.')

if new_r2 > baseline_r2:
    print('The new model improves on the baseline model in terms of R^2.')
else:
    print('The new model does not provide a significant improvement over the baseline model in terms of R^2.')


In [ ]:
## Basic Linear Regression

In [ ]:
## Polynomial Regression

### Regression Results

In [ ]:
# Fit the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Get the coefficients
coefficients = model.coef_

# Identify the features with strong relationships to sale prices
feature_names = X.columns
strong_features = []
for feature, coefficient in zip(feature_names, coefficients):
    if abs(coefficient) > 0.5:  # Adjust the threshold as needed
        strong_features.append((feature, coefficient))

# Sort the features by their coefficients
strong_features.sort(key=lambda x: abs(x[1]), reverse=True)

# Print the strong features and their coefficients
for feature, coefficient in strong_features:
    print(f"{feature}: {coefficient}")

# Explain the implications of the results
print("Implications:")
print("The following features have a strong relationship with sale prices:")
for feature, coefficient in strong_features:
    print(f"- {feature}")
print("This means that these features have a significant impact on the sale prices of the properties.")

# Identify specific actions for stakeholders
print("Actions for stakeholders:")
print("Based on these results, stakeholders can:")
print("- Focus on improving the quality of the properties, as 'condition_Fair' has a positive impact on sale prices.")
print("- Invest in properties with a waterfront view, as 'waterfront_YES' has a strong positive impact on sale prices.")
print("- Consider properties with larger living areas ('sqft_living'), as it has a strong positive impact on sale prices.")


## Data Visualization

Here we will be using Exploratory Data Analysis to allow shareholders understand the value and success of how different features affect prices.

In [ ]:
# creating a scatter plot between square footage and sales price
# Create a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='sqft_living', y='price', hue='condition', data=data_encoded)

# Add labels and title
plt.xlabel('Square Footage of Living Area')
plt.ylabel('Sale Price')
plt.title('Relationship between Square Footage and Sale Price')

# Add a legend
#plt.legend(title='Condition')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Create a box plot
plt.figure(figsize=(10, 6))
sns.boxplot(x='condition', y='price', data=data)

# Add labels and title
plt.xlabel('Condition')
plt.ylabel('Sale Price')
plt.title('Distribution of Sale Prices by Condition')

# Add additional annotations or text if needed
# For example, you can add median price labels
medians = data.groupby('condition')['price'].median()
for i, median in enumerate(medians):
    plt.text(i, median, f'Median: ${median}', ha='center')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate average sale price for each year
avg_price_by_year = data.groupby('yr_built')['price'].mean()

# Create a line plot
plt.figure(figsize=(12, 6))
sns.lineplot(x=avg_price_by_year.index, y=avg_price_by_year.values)

# Add labels and title
plt.xlabel('Year Built')
plt.ylabel('Average Sale Price')
plt.title('Trend of Sale Prices Over Time')

# Adjust x-axis ticks and labels
plt.xticks(rotation=45)

# Show the plot
plt.show()
